# ETL Project
## Terrence Cummings
Scraping race mix by Minneapolis neighborhood

In [376]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import json
import requests
import pymongo
from splinter import Browser
from selenium import webdriver
import time
import sys
import os
import datetime
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.add_argument("--headless")

In [377]:
#Read in CSV's from Extract
cmnty_links_df=pd.read_csv('cmnty_links.csv')
nbhd_links_df=pd.read_csv('nbhd_links.csv')
cmnty_race_df=pd.read_csv('cmnty_race.csv')
nbhd_race_df=pd.read_csv('nbhd_race.csv')
cmnty_keys_df=pd.read_csv('cmnty_keys.csv')
nbhd_keys_df=pd.read_csv('nbhd_keys.csv')

In [378]:
nbhd_race_df = nbhd_race_df.replace('suppressed', np.nan)


In [379]:
nbhd_race_df['white_cnt'] = nbhd_race_df['white_cnt'].str.replace(',', '').astype(float)
nbhd_race_df['black_cnt'] = nbhd_race_df['black_cnt'].str.replace(',', '').astype(float)
nbhd_race_df['native_cnt'] = nbhd_race_df['native_cnt'].str.replace(',', '').astype(float)
nbhd_race_df['asian_cnt'] = nbhd_race_df['asian_cnt'].str.replace(',', '').astype(float)
nbhd_race_df['other_cnt'] = nbhd_race_df['other_cnt'].str.replace(',', '').astype(float)
nbhd_race_df['two_or_more_cnt'] = nbhd_race_df['two_or_more_cnt'].str.replace(',', '').astype(float)
nbhd_race_df['hispanic_cnt'] = nbhd_race_df['hispanic_cnt'].str.replace(',', '').astype(float)
nbhd_race_df['of_color_cnt'] = nbhd_race_df['of_color_cnt'].str.replace(',', '').astype(float)

In [380]:
nbhd_race_df['white_pct'] = nbhd_race_df['white_pct'].str.replace('%', '').astype(float)/100
nbhd_race_df['black_pct'] = nbhd_race_df['black_pct'].str.replace('%', '').astype(float)/100
nbhd_race_df['native_pct'] = nbhd_race_df['native_pct'].str.replace('%', '').astype(float)/100
nbhd_race_df['asian_pct'] = nbhd_race_df['asian_pct'].str.replace('%', '').astype(float)/100
nbhd_race_df['other_pct'] = nbhd_race_df['other_pct'].str.replace('%', '').astype(float)/100
nbhd_race_df['two_or_more_pct'] = nbhd_race_df['two_or_more_pct'].str.replace('%', '').astype(float)/100
nbhd_race_df['hispanic_pct'] = nbhd_race_df['hispanic_pct'].str.replace('%', '').astype(float)/100
nbhd_race_df['of_color_pct'] = nbhd_race_df['of_color_pct'].str.replace('%', '').astype(float)/100

In [381]:
nbhd_race_df['total_cnt']=round(nbhd_race_df['white_cnt']/nbhd_race_df['white_pct'],0)

In [382]:
temp_nbhd_race_df = nbhd_race_df.merge(nbhd_keys_df, how='outer', left_on='neighborhood', right_on='neighborhood_url')
final_nbhd_race_df=temp_nbhd_race_df.drop(['neighborhood', 'neighborhood_url'], axis = 1) 
final_nbhd_race_df.set_index('NBHD_KEY', inplace=True)


In [383]:
final_nbhd_race_df = final_nbhd_race_df[['NEIGHBORHOOD','COMMUNITY','total_cnt','white_cnt','white_pct','black_cnt','black_pct','native_cnt','native_pct','asian_cnt','asian_pct','other_cnt','other_pct','two_or_more_cnt','two_or_more_pct','hispanic_cnt','hispanic_pct','of_color_cnt','of_color_pct','URL']]


In [384]:
final_nbhd_race_df

,NEIGHBORHOOD,COMMUNITY,total_cnt,white_cnt,white_pct,black_cnt,black_pct,native_cnt,native_pct,asian_cnt,asian_pct,other_cnt,other_pct,two_or_more_cnt,two_or_more_pct,hispanic_cnt,hispanic_pct,of_color_cnt,of_color_pct,URL
NBHD_KEY,,,,,,,,,,,,,,,,,,,,
N71,ARMATAGE,SOUTHWEST,5108.0,4316.0,0.845,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,185.0,0.036,NaN,NaN,792.0,0.155,http://www.mncompass.org/profiles/neighborhood...
N46,AUDUBON PARK,NORTHEAST,5119.0,4034.0,0.788,386.0,0.075,NaN,NaN,NaN,NaN,NaN,NaN,303.0,0.059,449.0,0.088,1083.0,0.212,http://www.mncompass.org/profiles/neighborhood...
N63,BANCROFT,POWDERHORN,3544.0,2601.0,0.734,413.0,0.116,NaN,NaN,74.0,0.021,NaN,NaN,278.0,0.078,521.0,0.147,943.0,0.266,http://www.mncompass.org/profiles/neighborhood...
N02,CEDAR-ISLES-DEAN,CALHOUN-ISLE,3195.0,2927.0,0.916,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.mncompass.org/profiles/neighborhood...
N80,CEDAR RSIDE/WEST BANK,UNIVERSITY,10059.0,3219.0,0.320,5146.0,0.511,NaN,NaN,1156.0,0.115,NaN,NaN,396.0,0.039,270.0,0.027,6846.0,0.680,http://www.mncompass.org/profiles/neighborhood...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
N55,ST ANTHONY EAST,NORTHEAST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
N56,ST ANTHONY WEST,NORTHEAST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
N33,SUMNER-GLENWOOD,NEAR NORTH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [385]:
cmnty_race_df['community'] = cmnty_race_df['community'].str.upper()
cmnty_race_df = cmnty_race_df.replace('suppressed', '0')
cmnty_race_df = cmnty_race_df.replace(np.nan, '0', regex=True)


In [386]:
cmnty_race_df['white_cnt'] = cmnty_race_df['white_cnt'].str.replace(',', '').astype(float)
cmnty_race_df['black_cnt'] = cmnty_race_df['black_cnt'].str.replace(',', '').astype(float)
cmnty_race_df['native_cnt'] = cmnty_race_df['native_cnt'].str.replace(',', '').astype(float)
cmnty_race_df['asian_cnt'] = cmnty_race_df['asian_cnt'].str.replace(',', '').astype(float)
cmnty_race_df['other_cnt'] = cmnty_race_df['other_cnt'].str.replace(',', '').astype(float)
cmnty_race_df['two_or_more_cnt'] = cmnty_race_df['two_or_more_cnt'].str.replace(',', '').astype(float)
cmnty_race_df['hispanic_cnt'] = cmnty_race_df['hispanic_cnt'].str.replace(',', '').astype(float)
cmnty_race_df['of_color_cnt'] = cmnty_race_df['of_color_cnt'].str.replace(',', '').astype(float)


In [387]:
cmnty_race_df['white_pct'] = cmnty_race_df['white_pct'].str.replace('%', '').astype(float)/100
cmnty_race_df['black_pct'] = cmnty_race_df['black_pct'].str.replace('%', '').astype(float)/100
cmnty_race_df['native_pct'] = cmnty_race_df['native_pct'].str.replace('%', '').astype(float)/100
cmnty_race_df['asian_pct'] = cmnty_race_df['asian_pct'].str.replace('%', '').astype(float)/100
cmnty_race_df['other_pct'] = cmnty_race_df['other_pct'].str.replace('%', '').astype(float)/100
cmnty_race_df['two_or_more_pct'] = cmnty_race_df['two_or_more_pct'].str.replace('%', '').astype(float)/100
cmnty_race_df['hispanic_pct'] = cmnty_race_df['hispanic_pct'].str.replace('%', '').astype(float)/100
cmnty_race_df['of_color_pct'] = cmnty_race_df['of_color_pct'].str.replace('%', '').astype(float)/100

In [388]:
cmnty_dict = {'CALHOUN-ISLES': 'CALHOUN-ISLE', 'CAMDEN':'CAMDEN', 'CENTRAL': 'CENTRAL', 'LONGFELLOW': 'LONGFELLOW', 'NEAR-NORTH': 'NEAR NORTH', 'NOKOMIS':'NOKOMIS', 'NORTHEAST':'NORTHEAST', 'PHILLIPS': 'PHILLIPS', 'POWDERHORN': 'POWDERHORN', 'SOUTHWEST':'SOUTHWEST', 'UNIVERSITY':'UNIVERSITY'}

In [389]:
cmnty_race_df = cmnty_race_df.replace({"community": cmnty_dict})

In [390]:
final_cmnty_race_df = cmnty_race_df.merge(cmnty_keys_df, how='left', left_on='community', right_on='COMMUNITY')
del final_cmnty_race_df['COMMUNITY']
final_cmnty_race_df.set_index('CMNTY_KEY', inplace=True)
final_cmnty_race_df['total_cnt']=final_cmnty_race_df['white_cnt']+final_cmnty_race_df['of_color_cnt']


In [391]:
final_cmnty_race_df['white_pct']=final_cmnty_race_df['white_cnt']/final_cmnty_race_df['total_cnt']
final_cmnty_race_df['black_pct']=final_cmnty_race_df['black_cnt']/final_cmnty_race_df['total_cnt']
final_cmnty_race_df['native_pct']=final_cmnty_race_df['native_cnt']/final_cmnty_race_df['total_cnt']
final_cmnty_race_df['asian_pct']=final_cmnty_race_df['asian_cnt']/final_cmnty_race_df['total_cnt']
final_cmnty_race_df['other_pct']=final_cmnty_race_df['other_cnt']/final_cmnty_race_df['total_cnt']
final_cmnty_race_df['two_or_more_pct']=final_cmnty_race_df['two_or_more_cnt']/final_cmnty_race_df['total_cnt']
final_cmnty_race_df['hispanic_pct']=final_cmnty_race_df['hispanic_cnt']/final_cmnty_race_df['total_cnt']
final_cmnty_race_df['of_color_pct']=final_cmnty_race_df['of_color_cnt']/final_cmnty_race_df['total_cnt']
final_cmnty_race_df = final_cmnty_race_df[['community','total_cnt','white_cnt','white_pct','black_cnt','black_pct','native_cnt','native_pct','asian_cnt','asian_pct','other_cnt','other_pct','two_or_more_cnt','two_or_more_pct','hispanic_cnt','hispanic_pct','of_color_cnt', 'of_color_pct','URL']]
final_cmnty_race_df


,community,total_cnt,white_cnt,white_pct,black_cnt,black_pct,native_cnt,native_pct,asian_cnt,asian_pct,other_cnt,other_pct,two_or_more_cnt,two_or_more_pct,hispanic_cnt,hispanic_pct,of_color_cnt,of_color_pct,URL
CMNTY_KEY,,,,,,,,,,,,,,,,,,,
C00,CALHOUN-ISLE,32423.0,27911.0,0.860840,1591.0,0.049070,0.0,0.000000,1080.0,0.033310,416.0,0.012830,1309.0,0.040373,1166.0,0.035962,4512.0,0.139160,http://www.mncompass.org/profiles/communities/...
C01,CAMDEN,30729.0,13565.0,0.441440,10677.0,0.347457,343.0,0.011162,3500.0,0.113899,691.0,0.022487,1953.0,0.063556,2360.0,0.076800,17164.0,0.558560,http://www.mncompass.org/profiles/communities/...
C02,CENTRAL,35312.0,23384.0,0.662211,7133.0,0.201999,405.0,0.011469,2410.0,0.068249,410.0,0.011611,1564.0,0.044291,1645.0,0.046585,11928.0,0.337789,http://www.mncompass.org/profiles/communities/...
C03,LONGFELLOW,29012.0,20399.0,0.703123,5521.0,0.190301,290.0,0.009996,815.0,0.028092,901.0,0.031056,1072.0,0.036950,2083.0,0.071798,8613.0,0.296877,http://www.mncompass.org/profiles/communities/...
C04,NEAR NORTH,35868.0,8264.0,0.230400,17163.0,0.478505,629.0,0.017537,5493.0,0.153145,1864.0,0.051968,2448.0,0.068250,3784.0,0.105498,27604.0,0.769600,http://www.mncompass.org/profiles/communities/...
C05,NOKOMIS,38790.0,30885.0,0.796210,3507.0,0.090410,629.0,0.016216,776.0,0.020005,1214.0,0.031297,1779.0,0.045862,2160.0,0.055684,7905.0,0.203790,http://www.mncompass.org/profiles/communities/...
C06,NORTHEAST,37473.0,28051.0,0.748566,3921.0,0.104635,674.0,0.017986,1035.0,0.027620,1999.0,0.053345,1793.0,0.047848,3989.0,0.106450,9422.0,0.251434,http://www.mncompass.org/profiles/communities/...
C07,PHILLIPS,22078.0,6310.0,0.285805,9606.0,0.435094,1255.0,0.056844,757.0,0.034288,3400.0,0.153999,750.0,0.033970,5709.0,0.258583,15768.0,0.714195,http://www.mncompass.org/profiles/communities/...
C08,POWDERHORN,57176.0,33738.0,0.590073,11305.0,0.197723,787.0,0.013765,1387.0,0.024258,7056.0,0.123408,2876.0,0.050301,13036.0,0.227998,23438.0,0.409927,http://www.mncompass.org/profiles/communities/...


In [392]:
nbhd_cmnty_grp_df = pd.DataFrame(final_nbhd_race_df.groupby('COMMUNITY').sum())

In [393]:
# nbhd_cmnty_grp_df=nbhd_cmnty_grp_df.drop(['white_pct','black_pct','native_pct','asian_pct','other_pct','two_or_more_pct','hispanic_pct','of_color_pct'], axis = 1) 
nbhd_cmnty_grp_df['total_cnt'] = nbhd_cmnty_grp_df['white_cnt']+nbhd_cmnty_grp_df['of_color_cnt']

In [394]:
nbhd_cmnty_grp_df['white_pct']=nbhd_cmnty_grp_df['white_cnt']/nbhd_cmnty_grp_df['total_cnt']
nbhd_cmnty_grp_df['black_pct']=nbhd_cmnty_grp_df['black_cnt']/nbhd_cmnty_grp_df['total_cnt']
nbhd_cmnty_grp_df['native_pct']=nbhd_cmnty_grp_df['native_cnt']/nbhd_cmnty_grp_df['total_cnt']
nbhd_cmnty_grp_df['asian_pct']=nbhd_cmnty_grp_df['asian_cnt']/nbhd_cmnty_grp_df['total_cnt']
nbhd_cmnty_grp_df['other_pct']=nbhd_cmnty_grp_df['other_cnt']/nbhd_cmnty_grp_df['total_cnt']
nbhd_cmnty_grp_df['two_or_more_pct']=nbhd_cmnty_grp_df['two_or_more_cnt']/nbhd_cmnty_grp_df['total_cnt']
nbhd_cmnty_grp_df['hispanic_pct']=nbhd_cmnty_grp_df['hispanic_cnt']/nbhd_cmnty_grp_df['total_cnt']
nbhd_cmnty_grp_df['of_color_pct']=nbhd_cmnty_grp_df['of_color_cnt']/nbhd_cmnty_grp_df['total_cnt']
nbhd_cmnty_grp_df = nbhd_cmnty_grp_df[['total_cnt','white_cnt','white_pct','black_cnt','black_pct','native_cnt','native_pct','asian_cnt','asian_pct','other_cnt','other_pct','two_or_more_cnt','two_or_more_pct','hispanic_cnt','hispanic_pct','of_color_cnt', 'of_color_pct']]

In [395]:
temp_cmnty_join_df = final_cmnty_race_df.merge(nbhd_cmnty_grp_df, how='left', left_on='community', right_index=True)

In [396]:
temp_cmnty_join_df['white_bal_pct'] = (temp_cmnty_join_df['total_cnt_x']*temp_cmnty_join_df['white_pct_x']-temp_cmnty_join_df['total_cnt_y']*temp_cmnty_join_df['white_pct_y'])/(temp_cmnty_join_df['total_cnt_x']-temp_cmnty_join_df['total_cnt_y'])

temp_cmnty_join_df['black_bal_pct'] = (temp_cmnty_join_df['total_cnt_x']*temp_cmnty_join_df['black_pct_x']-temp_cmnty_join_df['total_cnt_y']*temp_cmnty_join_df['black_pct_y'])/(temp_cmnty_join_df['total_cnt_x']-temp_cmnty_join_df['total_cnt_y'])

temp_cmnty_join_df['native_bal_pct'] = (temp_cmnty_join_df['total_cnt_x']*temp_cmnty_join_df['native_pct_x']-temp_cmnty_join_df['total_cnt_y']*temp_cmnty_join_df['native_pct_y'])/(temp_cmnty_join_df['total_cnt_x']-temp_cmnty_join_df['total_cnt_y'])

temp_cmnty_join_df['asian_bal_pct'] = (temp_cmnty_join_df['total_cnt_x']*temp_cmnty_join_df['asian_pct_x']-temp_cmnty_join_df['total_cnt_y']*temp_cmnty_join_df['asian_pct_y'])/(temp_cmnty_join_df['total_cnt_x']-temp_cmnty_join_df['total_cnt_y'])

temp_cmnty_join_df['other_bal_pct'] = (temp_cmnty_join_df['total_cnt_x']*temp_cmnty_join_df['other_pct_x']-temp_cmnty_join_df['total_cnt_y']*temp_cmnty_join_df['other_pct_y'])/(temp_cmnty_join_df['total_cnt_x']-temp_cmnty_join_df['total_cnt_y'])

temp_cmnty_join_df['two_or_more_bal_pct'] = (temp_cmnty_join_df['total_cnt_x']*temp_cmnty_join_df['two_or_more_pct_x']-temp_cmnty_join_df['total_cnt_y']*temp_cmnty_join_df['two_or_more_pct_y'])/(temp_cmnty_join_df['total_cnt_x']-temp_cmnty_join_df['total_cnt_y'])

temp_cmnty_join_df['hispanic_bal_pct'] = (temp_cmnty_join_df['total_cnt_x']*temp_cmnty_join_df['hispanic_pct_x']-temp_cmnty_join_df['total_cnt_y']*temp_cmnty_join_df['hispanic_pct_y'])/(temp_cmnty_join_df['total_cnt_x']-temp_cmnty_join_df['total_cnt_y'])

temp_cmnty_join_df['of_color_bal_pct'] = (temp_cmnty_join_df['total_cnt_x']*temp_cmnty_join_df['of_color_pct_x']-temp_cmnty_join_df['total_cnt_y']*temp_cmnty_join_df['of_color_pct_y'])/(temp_cmnty_join_df['total_cnt_x']-temp_cmnty_join_df['total_cnt_y'])

temp_cmnty_join_df['marg_err']=(temp_cmnty_join_df['total_cnt_x']-temp_cmnty_join_df['total_cnt_y'])/temp_cmnty_join_df['total_cnt_x']

In [397]:
temp_cmnty_join_df=temp_cmnty_join_df.drop(['total_cnt_x','white_cnt_x', 'black_cnt_x', 'native_cnt_x','asian_cnt_x','other_cnt_x','two_or_more_cnt_x','hispanic_cnt_x','of_color_cnt_x','total_cnt_y','white_cnt_y','black_cnt_y','native_cnt_y','asian_cnt_y','other_cnt_y','two_or_more_cnt_y','hispanic_cnt_y','of_color_cnt_y','URL'] , axis = 1) 


In [398]:
temp_cmnty_join_df.rename(columns={'white_pct_x': 'cmnty_white_pct', 'black_pct_x': 'cmnty_black_pct', 'native_pct_x': 'cmnty_native_pct', 'asian_pct_x': 'cmnty_asian_pct', 'other_pct_x': 'cmnty_other_pct', 'two_or_more_pct_x': 'cmnty_two_or_more_pct', 'hispanic_pct_x': 'cmnty_hispanic_pct', 'of_color_pct_x': 'cmnty_of_color_pct', 'white_pct_y': 'nbhd_white_pct', 'black_pct_y': 'nbhd_black_pct', 'native_pct_y': 'nbhd_native_pct', 'asian_pct_y': 'nbhd_asian_pct', 'other_pct_y': 'nbhd_other_pct', 'two_or_more_pct_y': 'nbhd_two_or_more_pct', 'hispanic_pct_y': 'nbhd_hispanic_pct', 'of_color_pct_y': 'nbhd_of_color_pct'}, inplace=True)

In [399]:
race_mix_nan_df = temp_cmnty_join_df
race_mix_nan_df

,community,cmnty_white_pct,cmnty_black_pct,cmnty_native_pct,cmnty_asian_pct,cmnty_other_pct,cmnty_two_or_more_pct,cmnty_hispanic_pct,cmnty_of_color_pct,nbhd_white_pct,...,nbhd_of_color_pct,white_bal_pct,black_bal_pct,native_bal_pct,asian_bal_pct,other_bal_pct,two_or_more_bal_pct,hispanic_bal_pct,of_color_bal_pct,marg_err
CMNTY_KEY,,,,,,,,,,,,,,,,,,,,,
C00,CALHOUN-ISLE,0.860840,0.049070,0.000000,0.033310,0.012830,0.040373,0.035962,0.139160,0.867721,...,0.132279,0.843526,0.025483,0.000000,0.036218,0.045110,0.064086,0.055736,0.156474,0.284428
C01,CAMDEN,0.441440,0.347457,0.011162,0.113899,0.022487,0.063556,0.076800,0.558560,0.457291,...,0.542709,0.405851,0.368571,0.036223,0.156828,0.072975,0.054599,0.121027,0.594149,0.308145
C02,CENTRAL,0.662211,0.201999,0.011469,0.068249,0.011611,0.044291,0.046585,0.337789,0.666189,...,0.333811,0.587709,0.226816,0.086592,0.119553,0.229050,0.035196,0.032961,0.412291,0.050691
C03,LONGFELLOW,0.703123,0.190301,0.009996,0.028092,0.031056,0.036950,0.071798,0.296877,0.703133,...,0.296867,1.000000,-36.000000,-245.000000,-189.000000,-374.000000,-0.000000,-412.000000,-0.000000,-0.000034
C04,NEAR NORTH,0.230400,0.478505,0.017537,0.153145,0.051968,0.068250,0.105498,0.769600,0.209381,...,0.790619,0.344636,0.397919,0.076247,0.092573,0.092573,0.080732,0.123789,0.655364,0.155403
C05,NOKOMIS,0.796210,0.090410,0.016216,0.020005,0.031297,0.045862,0.055684,0.203790,0.785304,...,0.214696,0.811247,0.102287,0.038572,0.030294,0.054271,0.052002,0.048445,0.188753,0.420392
C06,NORTHEAST,0.748566,0.104635,0.017986,0.027620,0.053345,0.047848,0.106450,0.251434,0.756839,...,0.243161,0.734049,0.149074,0.022493,0.061600,0.092546,0.044031,0.115922,0.265951,0.363035
C07,PHILLIPS,0.285805,0.435094,0.056844,0.034288,0.153999,0.033970,0.258583,0.714195,0.299936,...,0.700064,0.235886,0.297680,0.178403,0.110244,0.217614,0.022583,0.338945,0.764114,0.220627
C08,POWDERHORN,0.590073,0.197723,0.013765,0.024258,0.123408,0.050301,0.227998,0.409927,0.601738,...,0.398262,0.394532,0.279590,0.244486,0.031687,0.377446,0.054675,0.271824,0.605468,0.056300
